# Max Flow LP

### Author: [Ben Rosenberg](https://benrosenberg.info)

### Imports

We begin by importing some relevant libraries. We import `ortools` to formulate and solve the LP, and we import `time` to time the entire process of supplying the constraints solving the LP.

In [18]:
from ortools.linear_solver import pywraplp as OR
import time

### Input data

Next, we define our input data. Recall that in the Max Flow problem we have as an input a graph $G = (N, E)$, where $N$ is the set of nodes, and $E$ is the set of edges, each of which has a capacity $u(i,j)$ for all $(i,j)\in E$.

The input data below corresponds to a small example graph, seen below, in which the numbers inside the nodes are arbitrary indices, and the number corresponding to each edge denotes the capacity of that edge:

![](max_flow_graph.png)

We will create and solve an LP to determine the max flow from source node 0 to sink node 5.

In [19]:
nodes = range(6)

edges = [(0,1), (0,3), (1,2), (1,3), (2,3), (2,5), (3,4), (4,5)]

# capacity[i,j] is the capacity of edge (i,j)
capacity = {
    (0,1) : 5, 
    (0,3) : 3, 
    (1,2) : 3, 
    (1,3) : 1, 
    (2,3) : 2, 
    (2,5) : 6, 
    (3,4) : 4, 
    (4,5) : 3
}

### Model Definition

Now we define our model. The Max Flow problem has the following constraints, using the variable $f(i,j)$ to denote the amount of flow on edge $(i,j)$:

 - Net flow (out minus in) is equal to 0 for all nodes (except source and sink)
 $$\sum_{(j,i)\in E} f(j,i) - \sum_{(i,j)\in E} f(i,j) = 0 \quad \forall i\in N \backslash \{0,5\}$$
 - Flow on an edge can be no more than the capacity of that edge
 $$f(i,j) \leq u(i,j) \quad \forall (i,j)\in E$$
 - Flow on an edge can be no less than 0 units 
 $$f(i,j) \geq 0 \quad \forall (i,j) \in E$$
 
And the objective should be intuitive, as it's simply maximizing the total flow through the graph (which is equivalent to maximizing the flow into the sink node, $5$):

$$\max \sum_{(i,5)\in E} f(i,5)$$

In [20]:
start_time = time.time()

m = OR.Solver('Max Flow', OR.Solver.GLOP_LINEAR_PROGRAMMING)

# add variable f
f = {}
for (i,j) in edges:
    f[i,j] = m.NumVar(0, m.infinity(), 'f[{},{}]'.format(
        i,j
    ))

# add constraint flow in/out for non-source/sink nodes
for i in nodes:
    if i not in (0,5):
        m.Add(sum(f[j,i] for (j,x) in edges if x == i) - 
              sum(f[i,j] for (x,j) in edges if x == i)
              == 0)

# add constraint on edge flows w.r.t. capacities
for (i,j) in edges:
    m.Add(f[i,j] <= capacity[i,j])

# add constraint on edge flows w.r.t. 0
for (i,j) in edges:
    m.Add(f[i,j] >= 0)
    
# set objective
m.Maximize(sum(f[i,j] for (i,j) in edges if j == 5))

m.Solve()

end_time = time.time()

diff = time.gmtime(end_time - start_time)
print('\n[Total time used: {} minutes, {} seconds]'.format(diff.tm_min, diff.tm_sec))

print('Objective:', m.Objective().Value())


[Total time used: 0 minutes, 0 seconds]
Objective: 6.0


So we have our optimal objective. The optimal solution associated with that, in terms of flows on edges, is given below:

In [21]:
# optimal solution
for (i,j) in edges:
    print('f[{},{}] = {}'.format(
        i, j, f[i,j].solution_value()
    ))

f[0,1] = 3.0
f[0,3] = 3.0
f[1,2] = 3.0
f[1,3] = 0.0
f[2,3] = 0.0
f[2,5] = 3.0
f[3,4] = 3.0
f[4,5] = 3.0
